<a href="https://colab.research.google.com/github/NailKhelifa/TUM_research_project/blob/main/discrete_similarity_consensus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture

In [ ]:
# Data preparation : construction of datasets that consists in the returns of 200 S&P500 stocks over the period 2001-2023

stock_symbols = ["AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "TSLA", "META", "BRK.B", "UNH", "JPM", "JNJ", "XOM", "V", "PG", "AVGO", "LLY", "HD", "MA", "CVX", "MRK", "ABBV", "PEP", "COST", "ADBE", "KO", "WMT", "BAC", "CSCO", "MCD", "TMO", "CRM", "ACN", "PFE", "NFLX", "CMCSA", "ABT", "LIN", "AMD", "ORCL", "DHR", "WFC", "DIS", "TXN", "PM", "CAT", "INTC", "NEE", "UNP", "INTU", "COP", "VZ", "LOW", "NKE", "QCOM", "UPS", "IBM", "BA", "BMY", "HON", "SPGI", "AMAT", "RTX", "AMGN", "GE", "SBUX", "GS", "MS", "DE", "NOW", "PLD", "MDT", "ELV", "ISRG", "BLK", "BKNG", "ADP", "MDLZ", "LMT", "T", "TJX", "AXP", "SCHW", "ADI", "CVS", "MMC", "GILD", "LRCX", "SYK", "VRTX", "C", "AMT", "ETN", "CI", "CB", "ZTS", "REGN", "SLB", "MO", "TMUS", "FI", "EOG", "BDX", "MU", "PGR", "SO", "BSX", "CME", "PANW", "PYPL", "EQIX", "DUK", "ITW", "KLAC", "SNPS", "CSX", "ATVI", "AON", "SHW", "CL", "NOC", "CDNS", "ICE", "APD", "FCX", "FDX", "TGT", "HUM", "WM", "MMM", "MCK", "MPC", "ORLY", "HCA", "NXPI", "USB", "EMR", "PXD", "PH", "PNC", "CMG", "MCO", "APH", "ROP", "MAR", "NSC", "GM", "F", "GD", "PSX", "MCHP", "FTNT", "CARR", "EW", "MSI", "AJG", "TT", "DXCM", "ADM", "TDG", "AZO", "VLO", "CCI", "PSA", "OXY", "SRE", "ECL", "TEL", "AIG", "ON", "PCAR", "MNST", "ANET", "CHTR", "GIS", "ADSK", "STZ", "NUE", "MSCI", "KMB", "CTAS", "COF", "TFC", "JCI", "AFL", "D", "HEC", "IDXX", "AEP", "WMB", "WELL", "MET", "EXC", "HLT", "IQV", "PAYX", "EL", "CTVA", "O", "TRV", "ROST"]

# First month data

start1 = "2023-01-01"
end1 = "2023-02-01"
data = pd.DataFrame(yf.download(tickers=stock_symbols, start=start1, end=end1))
X_1 = pd.DataFrame(np.log(data["Close"]/data["Open"]).transpose())
X_1 = X_1.dropna()

# second month data

start2 = "2023-02-01"
end2 = "2023-03-01"
data2 = pd.DataFrame(yf.download(tickers=stock_symbols, start=start2, end=end2))
X_2 = pd.DataFrame(np.log(data2["Close"]/data2["Open"]).transpose())
X_2 = X_2.dropna()
new_stock_list_2 = np.array(X_2.index)

# second month data

start3 = "2023-03-01"
end3 = "2023-04-01"
data3 = pd.DataFrame(yf.download(tickers=stock_symbols, start=start3, end=end3))
X_3 = pd.DataFrame(np.log(data3["Close"]/data3["Open"]).transpose())
X_3 = X_3.dropna()
new_stock_list_3 = np.array(X_3.index)


[*********************100%%**********************]  200 of 200 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['HEC', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  200 of 200 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['HEC', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  200 of 200 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['HEC', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [ ]:
def same_cluster(stock_1, stock_2, consensus_matrix):
  return int((consensus_matrix.loc[stock_1][stock_2] == 1 or consensus_matrix.loc[stock_1][stock_2] == 1))

In [ ]:
def consensus_clustering(n_repeat, data, n_components, cov_type):

  '''
  ## Type of data ##

  n_repeat : integer --> number of time we apply the clustering method
  model : sklearn model we use --> e.g. GaussianMixture()
  data : pd.DataFrame --> data we want to fit to the model

  -------------

  ## Output ##

  Outputs a pandas DataFrame object of shape (len(data.index), n_repeat)

  -------------

  %%%%% STEP 1 %%%%%%

  The idea is here to train the model on the dataset data multiple time (here n_repeat time)
  and create a DataFrame whose columns are the cluster labels of each stock and whose rows are
  the label of a given stock for each clustering method

  '''

  model = GaussianMixture(n_components=n_components, covariance_type=cov_type)

  Y = pd.DataFrame(index=data.index)

  for i in range(n_repeat):
    model.fit(data)
    predicted_labels = model.predict(data)
    data_with_clusters = pd.DataFrame(predicted_labels, index=data.index)
    y_i = "Clustering n°%i" % (i+1)
    Y[y_i] = data_with_clusters



  print("The first step is done")

  '''

   %%%%% STEP 2 %%%%%%

  Based on the output Y of multiple clusterings of the same dataset, create a
  symmetric consensus matrix with rows and columns corresponding to the symbols of
  the stocks we consider. The coefficients of this matrix have values 1 (if two stocks
  are in the same cluster in more than 75% of the realized clusterings given in Y) and
  0 if not.

  '''

  ############## We first compute the consensus_matrix ###############

  n_stocks = len(data.index) ## corresponds to the number of stocks in our database

  consensus_matrix = np.zeros((n_stocks, n_stocks))

  for i in range(n_stocks):
    for j in range(i, n_stocks):
      compteur = 0
      for k in range(n_repeat):
        if (Y.iloc[i,:][k] == Y.iloc[j,:][k]):
          compteur += 1
      if compteur >= 0.75*n_repeat:
        consensus_matrix[i][j] = 1
      else:
        consensus_matrix[i][j] = 0

  consensus_matrix = pd.DataFrame(consensus_matrix, index=data.index, columns=data.index).transpose()

  print("The second step is done")

  '''

  %%%%% STEP 3 %%%%%%

  Based on the consensus matrix that we just created, we compute the final clustering
  that corresponds to this consensus.

  '''

  ############### We then compute the final clustering from this consensus matrix ################

  S = data.index.tolist()
  clusters = np.array([])
  cluster_len = []
  nb_iterations = 0 ## way to count and control the number of iterations

  while len(S) > 0 and nb_iterations < n_components:
    stock_ref = S[0]
    cluster_i = np.array([])
    size_cluster_i = 0

    for stock in S :
      if (same_cluster(stock, stock_ref, consensus_matrix) == 1): ## we should not impose further conditions as the consensus relation should be transitive within a cluster
        cluster_i = np.append(cluster_i, np.array([stock]))
        size_cluster_i += 1

    cluster_len.append(size_cluster_i)

    clusters = np.hstack((clusters, cluster_i))

    S = np.setdiff1d(S, clusters) ## we remove the stocks that have already been attributed to a cluster
    nb_iterations += 1

  final_clustering = np.array(np.split(clusters, np.cumsum(cluster_len)[:-1]))
  index=["Cluster n°%i"%(i+1) for i in range(n_components)]

  print("The third step is done")


  return pd.DataFrame(final_clustering, index=index, columns=["Cluster composition n°1"])

In [ ]:
def same_cluster(stock_1, stock_2, consensus_matrix):
  return int((consensus_matrix.loc[stock_1][stock_2] == 1 or consensus_matrix.loc[stock_1][stock_2] == 1))

In [ ]:
## We create a dataframe of the cluster for the first dataset X_1 to have an idea of the clusters compositions

n_repeat = 15
n_components = 5
cov_type = 'spherical'
data = X_1

previous_cluster = consensus_clustering(n_repeat, data, n_components, cov_type)

# Given the composition of the cluster for X_1, we compute the average value of the stocks composing each cluster with their values in X_2
'''
we compute the average of the stock return during the second period for each cluster (given thecomposition we obtained at t-1).
'''
mean = []
for i in range(n_components): ## change the value 10 if the number of clusters is not the same
  labels_i = previous_cluster.iloc[i][0]
  mean_i = X_2.mean(axis=1)[labels_i].mean()
  mean.append(mean_i)

init_mean=np.zeros((n_components, 19))

for i in range(n_components): ## we range in the number of clusters (11) and for each cluster
  y_i = mean[i] * np.ones(19) ## there are 19 days !!
  init_mean[i] = y_i

init_mean = pd.DataFrame(init_mean, index=previous_cluster.index)

init_mean=init_mean

# means_initarray-like of shape (n_components, n_features),  here n_components = 11,  n_features : 198

model2 = GaussianMixture(n_components=n_components, covariance_type=cov_type, means_init=init_mean)

model2.fit(X_2)

Y_2 = pd.DataFrame(model2.predict(X_2), index=new_stock_list_2)

# We define a dataframe df_clusters_names_2 that gives the composition at each cluster for the methods applied to X_2

df_clusters_names_2 = pd.DataFrame(index = previous_cluster.index, columns = ["Cluster composition n°2"])
for i in range(5):
  g_i = Y_2[Y_2.iloc[:, 0] == i].index.tolist()
  df_clusters_names_2.iloc[i][0] = g_i

df_clusters_names_2

comparison_df = pd.concat([previous_cluster, df_clusters_names_2], axis=1)

The first step is done
The second step is done
The third step is done


<ipython-input-7-4ea8e5b8b404>:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final_clustering = np.array(np.split(clusters, np.cumsum(cluster_len)[:-1]))


In [ ]:
comparison_df

,Cluster composition n°1,Cluster composition n°2
Cluster n°1,"[AAPL, ACN, ADBE, ADSK, AVGO, AXP, CDNS, CMG, ...","[ABBV, ADM, ADP, AEP, AJG, AMGN, AMT, AON, APD..."
Cluster n°2,"[ABBV, ABT, AFL, AIG, AJG, AMGN, AZO, BAC, BDX...","[AAPL, AFL, AIG, ATVI, AXP, AZO, BA, BKNG, BMY..."
Cluster n°3,"[ADI, AEP, BLK, CARR, COST, CSCO, CSX, CTAS, E...","[ABT, ACN, ADI, AMZN, APH, BDX, BLK, BSX, CARR..."
Cluster n°4,"[ADM, CI, ELV, HUM, MRK, UNH, WMB]","[COP, CVX, DXCM, EOG, MPC, OXY, PSX, PXD, SLB,..."
Cluster n°5,[ADP],"[ADBE, ADSK, AMAT, AMD, ANET, AVGO, CDNS, DIS,..."


In [ ]:
# means_initarray-like of shape (n_components, n_features),  here n_components = 11,  n_features : 198

XX_3 = X_3.iloc[:, 0:19]

model3 = GaussianMixture(n_components=n_components, covariance_type=cov_type, means_init=model2.means_)

model3.fit(XX_3)

Y_3 = pd.DataFrame(model3.predict(XX_3), index=XX_3.index)

# We define a dataframe df_clusters_names_2 that gives the composition at each cluster for the methods applied to X_3

df_clusters_names_3 = pd.DataFrame(index = previous_cluster.index, columns = ["Cluster composition n°3"])
for i in range(5):
  g_i = Y_3[Y_3.iloc[:, 0] == i].index.tolist()
  df_clusters_names_3.iloc[i][0] = g_i


In [ ]:
comparison_df = pd.concat([previous_cluster, df_clusters_names_2, df_clusters_names_3], axis=1)
comparison_df

,Cluster composition n°1,Cluster composition n°2,Cluster composition n°3
Cluster n°1,"[AAPL, ACN, ADBE, ADI, ADP, ADSK, AEP, APD, AV...","[ADBE, ADSK, AMAT, AMD, ANET, DIS, ECL, EW, FT...","[APD, BA, CARR, COP, CVX, EOG, ETN, JCI, MPC, ..."
Cluster n°2,"[ABBV, ABT, AFL, AIG, AJG, AMGN, AZO, BAC, BDX...","[AFL, AIG, ATVI, BMY, CI, CME, COP, CTVA, CVX,...","[AIG, AMD, BAC, C, DXCM, FI, INTC, JPM, KLAC, ..."
Cluster n°3,"[ADM, CI, ELV, HUM, MRK, ORLY, PGR, TMUS, TRV,...","[ABBV, ADM, ADP, AEP, AJG, AMGN, AMT, AON, APD...","[ADP, APH, ATVI, AZO, BDX, BMY, BSX, CL, COST,..."
Cluster n°4,"[AMAT, AMD, AMZN, ANET, COF, FTNT, KLAC, LRCX,...","[AAPL, ADI, APH, AVGO, AXP, BA, BAC, BKNG, C, ...",[]
Cluster n°5,"[AMT, AON, ATVI, D, EL, MMC, SYK]","[ABT, ACN, AMZN, BDX, BLK, BSX, CARR, CHTR, CM...","[AAPL, ABBV, ABT, ACN, ADBE, ADI, ADM, ADSK, A..."


In [ ]:
model4 = GaussianMixture(n_components=n_components, covariance_type=cov_type)
model4.fit(X_1)
Y_1 = pd.DataFrame(model4.predict(X_1), index=X_1.index)
dir(model4)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_parameters',
 '_compute_lower_bound',
 '_e_step',
 '_estimate_log_prob',
 '_estimate_log_prob_resp',
 '_estimate_log_weights',
 '_estimate_weighted_log_prob',
 '_estimator_type',
 '_get_param_names',
 '_get_parameters',
 '_get_tags',
 '_initialize',
 '_initialize_parameters',
 '_m_step',
 '_more_tags',
 '_n_parameters',
 '_parameter_constraints',
 '_print_verbose_msg_init_beg',
 '_print_verbose_msg_init_end',
 '_print_verbose_msg_iter_end',
 '_repr_html_',
 '_repr_html_inner',
